In [2]:
import numpy as np
from scipy.optimize import minimize

In [102]:
n = 10
a = np.array([np.random.uniform(3, 7) for _ in range(n)])

t_i = lambda f_i, c_i, a_i: a_i + (f_i/c_i)**3
T = lambda f, c, a=a: np.sum(t_i(fi, ci, ai)*fi for fi, ci, ai in zip(f, c, a))

int_t_i = lambda f_i, c_i, a_i: a_i * f_i + (f_i)**3/(4 * c_i**3)
z = lambda f, c, a=a: np.sum(int_t_i(f_i, c_i, a_i) for f_i, c_i, a_i in zip(f, c, a))

In [103]:
def get_k_best(l, c, k=5):
    """
    Finds k best rows in l
    
    takes:
        l -- matrix of T_i, f, c
        k -- k best rows
    returns:
        k best rows
    """
    mat = l
    sor = c[np.argsort(mat)]
    return sor[:k]

In [104]:
def cross_breed(vects, k=5):
    """
    Computes cross breeing of vectors
    
    takes:
        vects      -- vectors, that new generation would be created from
        constrain  -- (float) size of new vector space (kinda)
        k          -- amount vectors that would be used to produce new child
        
    returns:
        new vector
    """
    coeff = np.random.dirichlet(np.ones(vects.shape[0]), size=1).reshape(-1)
    
    conv_mat = np.array([ci * vi for ci, vi in zip(coeff, vects)])
    new_vec = conv_mat.sum(axis=0)
    return new_vec

In [105]:
def gen_population(dim_pop, dim_c=n, C=150, costs=None, Ts = None):
    """
    Generates population. 
    On first step, when no c initialized, generates random c. 
    On next steps calculates c using cross of gens.
    
    takes: 
        dim_pop -- number of populations (how many rows in returned matrix)
        dim_c   -- dimention of population (how many columns in returned matrix)
        C       -- max cost sum (sum(c_i) <= C)
        costs   -- passed if it is desired to calculate not on first step
    returns:
        matrix c[dim_pop, dim_c]
    """
    c = np.zeros((dim_pop, dim_c))
    if costs is None:
        for i in range(dim_pop):
            c[i] = np.array([np.random.uniform(0, C/dim_c) for j in range(dim_c)])
    else:
        # number of used vects
        k = 5
        k_best = get_k_best(Ts, costs, k)
        for i in range(dim_pop):
            c[i] = cross_breed(k_best, k)
    return c

In [106]:
def mineze(c, F = 500, m=n):
    """
    Calculates step of inner minimization. 
    
    takes:
        c -- vector of costs
        F -- total flow of network
        m -- number of connections in graph
        
    returns:
        tuple: T_i, f, c
        T_i -- value in min point f
        f   -- found min with specified c
        c   -- passed c
    """
    cons = ({'type' : 'eq',  'fun' : lambda x: np.sum(x) - F}, 
            {'type' : 'ineq','fun' : lambda x: x            })
    f = np.ones(n) * F/m

    f = minimize(lambda x: z(x, c), f, constraints=cons).x
    T_i = T(f, c)
    return T_i

In [108]:
from sklearn.metrics import euclidean_distances

c = None
Tfc = None
T_prev = 0
prec = 1

epoch = 0
while prec > 1e-3:
    c = gen_population(10, costs=c, Ts=Tfc)
    Tfc = []
    for c_i in c:
        Tfc.append(mineze(c_i))

    ind_best = np.argsort(Tfc)[0]
    
    prec = euclidean_distances(Tfc[ind_best], T_prev)
    T_prev = Tfc[ind_best]
    epoch = epoch + 1
    
    print('epoch: {}'.format(epoch))
    print('T value: {}'.format(Tfc[ind_best]))
    print('costs: {}'.format(c[ind_best]))
    print('precision: {}\n'.format(prec))

epoch: 1
T value: 116823.88133550283
costs: [ 14.47013837  10.67310655  14.09931142  12.78562597   7.78850562
   7.81408469   2.83974404   5.5895527   11.58295638   7.69550401]
precision: [[ 116823.8813355]]

epoch: 2
T value: 135660.178351536
costs: [ 12.59591643  11.04111854  11.36377329  12.4942291    6.65469644
  10.23225581   3.00044562   4.49016832  10.17663034   5.48268593]
precision: [[ 18836.29701603]]

epoch: 3
T value: 140416.7158629783
costs: [ 11.53793975  10.4560223   10.20545779  12.63772126   6.31762423
  10.44237674   3.50447251   5.239545     9.53197337   4.58662412]
precision: [[ 4756.53751144]]

epoch: 4
T value: 141033.62743473833
costs: [ 10.75437866   9.7723178    9.48470717  12.68079753   6.23136149
  10.37635914   3.84911849   5.94773907   9.38272636   4.35527573]
precision: [[ 616.91157176]]

epoch: 5
T value: 141178.00125550746
costs: [ 10.96180554   9.98296957   9.78624015  12.69904434   6.25292142
  10.34272314   3.73908862   5.70063999   9.40689602   4.383